In [13]:
from pdf_gene import MoonCalc ,generate_pdf
from datetime import datetime
import requests
from tqdm import tqdm
import pandas as pd
from google import genai
import os
from dotenv import load_dotenv

In [14]:
def final_df(date,month, year, path="D:/code/Data_2035", dst='D:/Output'):
    
    date_obj = datetime.strptime(date, "%d-%m-%Y")    
    # Convert to YYYY-MM-DD format
    converted_date = date_obj.strftime("%Y-%m-%d")
    Moon = MoonCalc(path,converted_date,month,year +" AH",dst)
    return Moon.calculate()
final_df = final_df("30-03-2025","SHAWWAL","1446")

In [15]:
# Path to your .env file (outside the project folder)
load_dotenv(dotenv_path='../.env')

api_key = os.getenv("GEMINI_API_KEY")

In [16]:
df = final_df
# Split 'STATION(Sunset)' into 'Station' and 'SunsetTime'
df[['Station', 'SunsetTime']] = df['STATION(Sunset)'].str.extract(r'^(.*?)\s*\((.*?)\)$')

# Optional: drop the original column if no longer needed
df.drop(columns=['STATION(Sunset)'], inplace=True)

# Reorder columns if desired
df = df[['Station', 'SunsetTime', 'LAG TIME(Minutes)', 'MOON ALTITUDE(Degrees)',
         'SUN_AZIMUTH(Degrees)', 'DAZ(Degrees)', 'ELONGATION(Degrees)', 
         'ILLUMINATION(%)', 'CRITERION']]

row = df
row.columns = [
    'Station', 'SunsetTime', 'LagTime', 'MoonAltitude', 'SunAzimuth',
    'DAZ', 'Elongation', 'Illumination', 'Criterion'
]


In [17]:
visibility_mapping = {
    'A': 'Easily visible',
    'B': 'Visible under perfect conditions',
    'C': 'May need optical aid to find the crescent Moon',
    'D': 'Will need optical aid to find the crescent Moon',
    'E': 'Not visible with a telescope',
    'F': 'Not visible, below the Danjon limit'
}


In [ ]:
def make_prompt(row):
    # Visibility criterion description
    visibility_description = visibility_mapping.get(row['Criterion'], 'Unknown visibility criterion')

    # Interpret illumination
    illumination = float(row['Illumination'])
    illumination_description = (
        "The illumination is high, indicating good visibility."
        if illumination > 0.8 else
        "The illumination is low, which might make the Moon harder to see."
    )

    # Interpret moon altitude
    moon_altitude = float(row['MoonAltitude'])
    altitude_description = (
        "The Moon is high above the horizon, aiding visibility."
        if moon_altitude > 8 else
        "The Moon is low on the horizon, which may limit visibility."
    )

    return (
        f"On {row.name}, at station {row['Station']}, "
        f"moon altitude {moon_altitude}°, "
        f"illumination {illumination}%, "
        f"and visibility criterion '{row['Criterion']}': {visibility_description}. "
        f"{illumination_description} {altitude_description} "
        f"Based on this data, explain whether the moon is likely to be visible and why."
        f"Rate your confidence in your answer on a scale of 1-10 and explain briefly."
        f"Translate this response into Urdu."

    )

def query_gemini(prompt):

    client = genai.Client(api_key=api_key)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    return(response.text)

# ✅ Function to query LLaMA 3.2 via Ollama
def query_ollama(prompt, model="llama3.2"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"].strip()

tqdm.pandas()  # This enables tqdm integration with pandas

df['llama_response'] = df.progress_apply(lambda row: query_gemini(make_prompt(row)), axis=1)
print(df[['Station', 'llama_response']].head())



100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:57<00:00,  4.92s/it]

In [19]:
df["llama_response"]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for content in df["llama_response"]:
        print(content)
        print("------------xxxxxxxx----------")

Based on the data provided, the moon is **very likely to be visible** from Cherat at 2025-03-30 00:00:00. Here's a breakdown of why:

*   **Visibility Criterion 'A': Easily visible.** This is the most straightforward indicator. The observation explicitly states the moon is easily visible.
*   **Moon Altitude 14.0°:** An altitude of 14 degrees means the moon is well above the horizon. This elevation makes it easier to spot and less likely to be obscured by terrestrial objects like buildings or trees.
*   **Illumination 1.9%:** While the illumination is low (near a new moon), the statement emphasizes that the illumination is high. This is contradictory information. A low percentage like 1.9% means only a sliver of the moon is illuminated, but we are told that the illumination is high which would normally aid visibility, making the contradiction difficult to interpret.

**In conclusion:** Despite the contradiction of illumination percentage, the primary indicator of "Easily visible" and a

In [ ]:
# Sample DataFrame
# df = pd.DataFrame({
#     'name': ['Alice', 'Bob'],
#     'product': ['laptop', 'smartphone'],
#     'issue': ['screen not working', 'battery draining fast']
# })

df = final_df
# Generate a response per row
def generate_response(row):
    prompt = (
        f"Customer {row['name']} reported an issue with their {row['product']}: "
        f"'{row['issue']}'. Write a helpful and professional support message."
    )
    response = requests.post(
    'http://localhost:11434/api/generate',
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False
    }
    )
    return response.json()

# Apply model to DataFrame
df['llama_response'] = df.apply(generate_response, axis=1)

# Show results
print(df[['name', 'llama_response']])

In [ ]:
import pandas as pd


dfd = df

# Unpack the dictionary column
df_expanded = pd.json_normalize(dfd["llama_response"])

# Concatenate the unpacked data with the original DataFrame
df_new = pd.concat([dfd, df_expanded], axis=1).drop('llama_response', axis=1)

# Display the new DataFrame
print(df_new)

In [ ]:
df_new["response"]

In [ ]:
import pandas as pd
from llama_cpp import Llama

# Load the model (use the exact filename and path)
llm = Llama(model_path="llama-3.2-1B.Q4_K_M.gguf", n_ctx=2048)

# Sample DataFrame
df = pd.DataFrame({
    'name': ['Alice', 'Bob'],
    'product': ['laptop', 'smartphone'],
    'issue': ['screen not working', 'battery draining fast']
})

# Generate a response per row
def generate_response(row):
    prompt = (
        f"Customer {row['name']} reported an issue with their {row['product']}: "
        f"'{row['issue']}'. Write a helpful and professional support message."
    )
    output = llm(prompt, max_tokens=200)
    return output["choices"][0]["text"].strip()

# Apply model to DataFrame
df['llama_response'] = df.apply(generate_response, axis=1)

# Show results
print(df[['name', 'llama_response']])


In [ ]:
generate_pdf("30-03-2025", "SHAWWAL","1446")